In [1]:
#| default_exp updates
%load_ext autoreload
%autoreload 2

import sys
from pathlib import Path

# Insert in Path Project Directory
sys.path.insert(0, str(Path().cwd().parent))

# Atualização

> Este módulo atualiza as bases. Executa as queries sql do STEL, RADCOM e baixa os arquivos de estações e plano básico do MOSAICO.

In [2]:
#| export
from decimal import Decimal, getcontext
from typing import Union
import gc

import pandas as pd
import pyodbc
from rich.console import Console
from rich import print
from pyarrow import ArrowInvalid, ArrowTypeError
from fastcore.xtras import Path
from fastcore.foundation import L
from fastcore.test import test_eq
from tqdm.auto import tqdm
import pyodbc
from pymongo import MongoClient
from dotenv import load_dotenv

from extracao.constants import *
from extracao.format import parse_bw

getcontext().prec = 5
load_dotenv()

True

## Conexão com o banco de dados
A função a seguir é um `wrapper` simples que utiliza o `pyodbc` para se conectar ao banco de dados base da Anatel e retorna o objeto da conexão

In [3]:
#| export
def connect_db(
    server: str = "ANATELBDRO05",  # Servidor do Banco de Dados
    database: str = "SITARWEB",  # Nome do Banco de Dados
    trusted_conn: str = "yes",  # Conexão Segura: yes | no
    mult_results: bool = True,  # Múltiplos Resultados
) -> pyodbc.Connection:
    """Conecta ao Banco `server` e retorna o 'cursor' (iterador) do Banco"""
    return pyodbc.connect(
        "Driver={ODBC Driver 17 for SQL Server};"
        f"Server={server};"
        f"Database={database};"
        f"Trusted_Connection={trusted_conn};"
        f"MultipleActiveResultSets={mult_results};",
        timeout=TIMEOUT,
    )

In [4]:
#echo: false
def test_connection():
    conn = connect_db()
    cursor = conn.cursor()
    for query in (SQL_RADCOM,SQL_STEL):
        cursor.execute(query)
        test_eq(type(cursor.fetchone()), pyodbc.Row)

In [5]:
#| eval: false
test_connection()

In [6]:
#| export
def clean_mosaico(
    df: pd.DataFrame,  # DataFrame com os dados de Estações e Plano_Básico mesclados
    pasta: Union[
        str, Path
    ],  # Pasta com os dados de municípios para imputar coordenadas ausentes
) -> pd.DataFrame:  # DataFrame com os dados mesclados e limpos
    """Clean the merged dataframe with the data from the MOSAICO page"""
    df = df[
        df.Status.str.contains("-C1$|-C2$|-C3$|-C4$|-C7|-C98$", na=False)
    ].reset_index(drop=True)
    for c in df.columns:
        df.loc[df[c] == "", c] = pd.NA
    df.loc["Frequência"] = df.Frequência.astype("str").str.replace(",", ".")
    df = df[df.Frequência.notna()].reset_index(drop=True)
    df.loc["Frequência"] = df.Frequência.astype("float")
    df.loc[df.Num_Serviço == "205", "Frequência"] = df.loc[
        df.Num_Serviço == "205", "Frequência"
    ].apply(lambda x: Decimal(x) / Decimal(1000))
    df.loc[:, "Validade_RF"] = df.Validade_RF.astype("string").str.slice(0, 10)
    return df

## Atualização das bases de dados
As bases de dados são atualizadas atráves das funções a seguir, o único argumento passado em todas elas é a pasta na qual os arquivos locais processados serão salvos, os nomes dos arquivos são padronizados e não podem ser editados para que as funções de leitura e processamento recebam somente a pasta na qual esses arquivos foram salvos.

In [7]:
#| export
def _save_df(df: pd.DataFrame, folder: Union[str, Path], stem: str) -> pd.DataFrame:
    """Format, Save and return a dataframe"""
    df = df.copy() # Impedir a alteração do df original
    for c in df.columns:
        df[c] = df[c].astype("string").str.lstrip().str.rstrip()
    df = df.drop_duplicates(keep="first").reset_index(drop=True)
    if "Código_Município" in df:
        df = df[df.Código_Município.notna()].reset_index(drop=True)
    try:
        file = Path(f"{folder}/{stem}.parquet.gzip")
        df.to_parquet(file, compression="gzip", index=False)
    except (ArrowInvalid, ArrowTypeError) as e:        
            raise e(f"Não possível salvar o arquivo parquet {file}")
    return df

In [8]:
# | export
def update_radcom(
    conn: pyodbc.Connection,  # Objeto de conexão de banco
    folder: Union[str, Path],  # Pasta onde salvar os arquivos
) -> pd.DataFrame:  # DataFrame com os dados atualizados
    """Atualiza a tabela local retornada pela query `RADCOM`, com tratamento de erro de conectividade."""
    console = Console()
    with console.status(
        "[cyan]Lendo o Banco de Dados de Radcom...", spinner="monkey"
    ) as status:
        try:
            return _extract_radcom(conn, folder)
        except pyodbc.OperationalError as e:
            status.console.log(
                "Não foi possível abrir uma conexão com o SQL Server. Esta conexão somente funciona da rede cabeada!"
            )
            raise ConnectionError from e


def _extract_radcom(
    conn: pyodbc.Connection,  # Objeto de conexão de banco
    folder: Union[str, Path],  # Pasta onde salvar os arquivos
) -> pd.DataFrame:  # DataFrame com os dados atualizados
    df = pd.read_sql_query(SQL_RADCOM, conn)
    df["Entidade"] = df.Entidade.str.rstrip().str.lstrip()
    df["Num_Serviço"] = "231"
    df["Classe_Emissão"] = pd.NA
    df["Largura_Emissão(kHz)"] = "256"
    df["Validade_RF"] = pd.NA
    df["Status"] = "RADCOM"
    df["Fonte"] = "SRD"
    df["Multiplicidade"] = "1"
    a = df.Situação.isna()
    df.loc[a, "Classe"] = df.loc[a, "Fase"]
    df.loc[~a, "Classe"] = (
        df.loc[~a, "Fase"].astype("string")
        + "-"
        + df.loc[~a, "Situação"].astype("string")
    )
    df.drop(["Fase", "Situação"], axis=1, inplace=True)
    df = df.loc[:, COLUNAS]
    return _save_df(df, folder, "radcom")

In [9]:
#| eval: false
import warnings
import os
# warnings.filterwarnings("ignore", message='install "ipywidgets" for Jupyter support')
warnings.filterwarnings("ignore")

In [10]:
#| eval: false
folder = Path.cwd().parent / 'dados'
conn = connect_db()
radcom = update_radcom(conn, folder)
radcom

Output()

,Frequência,Entidade,Fistel,Número_Estação,Município,Código_Município,UF,Latitude,Longitude,Classe,Num_Serviço,Classe_Emissão,Largura_Emissão(kHz),Validade_RF,Status,Fonte,Multiplicidade
0,87.5,ASSOCIACAO COMUNITARIA VOZ DA LIBERDADE DE TUR...,50415095220,1008210959,Turilândia,2112456,MA,-2.228611111111,-45.306666666666665,P,231,<NA>,256,<NA>,RADCOM,SRD,1
1,87.5,ASSOCIACAO COMUNITARIA DO MORAD.DE ALVORADA DE...,50409064718,699491851,Alvorada de Minas,3102407,MG,-18.734166666666667,-43.36472222222217,3,231,<NA>,256,<NA>,RADCOM,SRD,1
2,87.5,ASSOCIAÇÃO COMUNITARIA DA JUVENTUDE DE CONGONH...,50405625782,699359830,Congonhas do Norte,3118106,MG,-18.812777777777665,-43.673611111111,3,231,<NA>,256,<NA>,RADCOM,SRD,1
3,87.5,ASSOCIACAO COMUNITARIA FOLHETA,50404381251,690859562,Dom Joaquim,3122603,MG,-18.95,-43.266666666666666,3,231,<NA>,256,<NA>,RADCOM,SRD,1
4,87.5,ASSOCIAÇÃO DE RÁDIO COMUNITÁRIA DE CASTANHEIRA...,50411566547,1008401606,Castanheira,5102850,MT,-11.137222222222167,-58.61333333333334,P-A,231,<NA>,256,<NA>,RADCOM,SRD,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4932,107.9,ASSOCIAÇÃO CULT COMUNIT MORUMBI,50407431578,697707695,São José dos Campos,3549904,SP,-23.267777777777667,-45.8975,3-B,231,<NA>,256,<NA>,RADCOM,SRD,1
4933,107.9,ASSOCIAÇÃO DE MORADORES DO JARDIM CRISTINA OUR...,50413097013,692270272,São José dos Campos,3549904,SP,-23.5530555555555,-45.8705555555555,3,231,<NA>,256,<NA>,RADCOM,SRD,1
4934,107.9,ASSOCIAÇÃO DE MORADORES DO JARDIM CRISTINA OUR...,50434484237,692270272,São José dos Campos,3549904,SP,-23.5530555555555,-45.8705555555555,3,231,<NA>,256,<NA>,RADCOM,SRD,1
4935,107.9,ASSOCIACAO COMUNITARIA CULTURAL DE MUSICA E CI...,50406778205,693049723,São José dos Campos,3549904,SP,-23.191944444444335,-45.87527777777767,3,231,<NA>,256,<NA>,RADCOM,SRD,1


In [11]:
#|export
def update_stel(
    conn: pyodbc.Connection,  # Objeto de conexão de banco
    folder: Union[str, Path],  # Pasta onde salvar os arquivos
) -> pd.DataFrame:  # DataFrame com os dados atualizados
    """Atualiza a tabela local retornada pela query `STEL`, com tratamento de erro de conectividade."""
    console = Console()
    with console.status(
        "[red]Lendo o Banco de Dados do STEL",
        spinner="grenade",
    ) as status:
        try:
            return _extract_stel(conn, folder)
        except pyodbc.OperationalError as e:
            status.console.log(
                "Não foi possível abrir uma conexão com o SQL Server. Esta conexão somente funciona da rede cabeada!"
            )
            raise ConnectionError from e


def _extract_stel(
    conn: pyodbc.Connection,  # Objeto de conexão de banco
    folder: Union[str, Path],  # Pasta onde salvar os arquivos
) -> pd.DataFrame:  # DataFrame com os dados atualizados
    """Atualiza a tabela local retornada pela query `STEL`"""
    stel = pd.read_sql_query(SQL_STEL, conn)
    stel["Status"] = "L"
    stel["Entidade"] = stel.Entidade.str.rstrip().str.lstrip()
    stel["Fonte"] = "STEL"
    stel.loc[:, ["Largura_Emissão(kHz)", "_"]] = (
        stel.Largura_Emissão.fillna("").apply(parse_bw).tolist()
    )
    stel.drop(["Largura_Emissão", "_"], axis=1, inplace=True)
    stel.loc[:, "Validade_RF"] = stel.Validade_RF.astype("string").str.slice(0, 10)
    stel.loc[stel.Unidade == "kHz", "Frequência"] = stel.loc[
        stel.Unidade == "kHz", "Frequência"
    ].apply(lambda x: Decimal(x) / Decimal(1000))
    stel.loc[stel.Unidade == "GHz", "Frequência"] = stel.loc[
        stel.Unidade == "GHz", "Frequência"
    ].apply(lambda x: Decimal(x) * Decimal(1000))
    stel.drop("Unidade", axis=1, inplace=True)
    stel["Multiplicidade"] = 1
    stel = stel.loc[:, COLUNAS]
    return _save_df(stel, folder, "stel")

In [12]:
#| eval: false
stel = update_stel(conn, folder)
stel

Output()

,Frequência,Entidade,Fistel,Número_Estação,Município,Código_Município,UF,Latitude,Longitude,Classe,Num_Serviço,Classe_Emissão,Largura_Emissão(kHz),Validade_RF,Status,Fonte,Multiplicidade
0,10512,GLOBO COMUNICAÇÃO E PARTICIPAÇÕES S/A,01032381230,87394,Rio de Janeiro,3304557,RJ,-22.951388888888832,-43.237222222222165,ML,256,F8W,18000.0,1997-03-05,L,STEL,1
1,10755,DIRECTNET PRESTACAO DE SERVICOS LTDA,50410115835,698520726,São Paulo,3550308,SP,-23.553825,-46.59835,FX,046,D7W,30000.0,2028-02-20,L,STEL,1
2,10755,BR27 SERVIÇOS DE TECNOLOGIA LTDA,50409073032,697961320,Vicência,2616308,PE,-7.611416666666667,-35.324861111111,FX,046,D7W,40000.0,2027-10-09,L,STEL,1
3,10835,DIRECTNET PRESTACAO DE SERVICOS LTDA,50410115835,698520467,São Paulo,3550308,SP,-23.619075,-46.73446666666667,FX,046,D7D,9000.0,2028-02-20,L,STEL,1
4,10835,JUNIOR E BRUNO PECAS E SERVICOS EM INFORMATICA...,50414049098,1003348782,Rio de Janeiro,3304557,RJ,-22.924988888888834,-43.634766666666664,FX,046,D7W,40000.0,2031-12-21,L,STEL,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68360,25.375,HOSPITAL NOSSA SENHORA DA CONCEICAO SA,50417771525,1008302110,Porto Alegre,4314902,RS,-30.010194444444334,-51.159388888888834,TX,060,F9W,25.0,2039-04-08,L,STEL,1
68361,25.375,REAL E BENEMERITA ASSOCIACAO PORTUGUESA DE BEN...,50404519970,1008765764,São Paulo,3550308,SP,-23.567138888888834,-46.64113888888883,TX,060,F9W,25.0,2037-05-08,L,STEL,1
68362,25.375,SOCIEDADE BENEFICENTE SAO CAMILO,50406930287,692391789,São Paulo,3550308,SP,-23.487777777777666,-46.627444444444336,TX,060,F9W,25.0,2040-05-05,L,STEL,1
68363,25.375,UNIMED PAULISTANA SOC COOPERATIVA DE TRABALHO ...,50407957189,696380056,São Paulo,3550308,SP,-23.562277777777666,-46.63858333333334,TX,060,F9W,25.0,2026-09-01,L,STEL,1


In [13]:
#|export
def update_mosaico(
    mongo_client: MongoClient,  # Objeto de conexão com o MongoDB
    folder: Union[str, Path],  # Pasta onde salvar os arquivos
) -> pd.DataFrame:  # DataFrame com os dados atualizados
    """Efetua a query na tabela de Radiodifusão no banco mongoDB `mongo_client` e atualiza o arquivo local"""
    console = Console()
    with console.status(
        "Consolidando os dados do Mosaico...", spinner="runner"
    ) as status:

        database = mongo_client["sms"]
        collection = database["srd"]
        list_data = list(collection.find(MONGO_SRD, projection=COLS_SRD.keys()))
        mosaico = pd.json_normalize(list_data)
        mosaico = mosaico.drop(columns=["estacao"])
        mosaico = mosaico[list(COLS_SRD.keys())]
        mosaico.rename(COLS_SRD, axis=1, inplace=True)
        mosaico = clean_mosaico(mosaico, folder)
        mosaico["Fonte"] = "MOS"
        mosaico['Num_Serviço'].fillna("", inplace=True)
        mosaico.loc[:, ["Largura_Emissão(kHz)", "Classe_Emissão"]] = (            
                mosaico.Num_Serviço.astype("string").fillna("").map(BW_MAP).apply(parse_bw).tolist()
        )
        mosaico.loc[mosaico.Classe_Emissão == "", "Classe_Emissão"] = pd.NA
        mosaico["Multiplicidade"] = 1
        mosaico = mosaico.loc[:, COLUNAS]
    return _save_df(mosaico, folder, "mosaico")

In [14]:
#|eval: false
uri = os.environ['MONGO_URI']
mongo_client = MongoClient(uri)
mongo_client.server_info()

{'version': '4.0.5',
 'gitVersion': '3739429dd92b92d1b0ab120911a23d50bf03c412',
 'targetMinOS': 'Windows 7/Windows Server 2008 R2',
 'modules': [],
 'allocator': 'tcmalloc',
 'javascriptEngine': 'mozjs',
 'sysInfo': 'deprecated',
 'versionArray': [4, 0, 5, 0],
 'openssl': {'running': 'Windows SChannel'},
 'buildEnvironment': {'distmod': '2008plus-ssl',
  'distarch': 'x86_64',
  'cc': 'cl: Microsoft (R) C/C++ Optimizing Compiler Version 19.00.24223 for x64',
  'ccflags': '/nologo /EHsc /W3 /wd4355 /wd4800 /wd4267 /wd4244 /wd4290 /wd4068 /wd4351 /wd4373 /we4013 /we4099 /we4930 /WX /errorReport:none /MD /O2 /Oy- /bigobj /utf-8 /Zc:rvalueCast /Zc:strictStrings /volatile:iso /Gw /Gy /Zc:inline',
  'cxx': 'cl: Microsoft (R) C/C++ Optimizing Compiler Version 19.00.24223 for x64',
  'cxxflags': '/TP',
  'linkflags': '/nologo /DEBUG /INCREMENTAL:NO /LARGEADDRESSAWARE /OPT:REF',
  'target_arch': 'x86_64',
  'target_os': 'windows'},
 'bits': 64,
 'debug': False,
 'maxBsonObjectSize': 16777216,
 '

In [15]:
#|eval: false
mosaico = update_mosaico(mongo_client, folder)
display(mosaico)

Output()

,Frequência,Entidade,Fistel,Número_Estação,Município,Código_Município,UF,Latitude,Longitude,Classe,Num_Serviço,Classe_Emissão,Largura_Emissão(kHz),Validade_RF,Status,Fonte,Multiplicidade
0,207.0,REDE DE COMUNICACOES ACREANA LTDA,50442889933,<NA>,Cruzeiro do Sul,1200203,AC,<NA>,<NA>,A,248,<NA>,6000.0,<NA>,TV-C1,MOS,1
1,539.0,X-MEDIAGROUP S.A.,50410887137,<NA>,Mâncio Lima,1200336,AC,<NA>,<NA>,C,248,<NA>,6000.0,<NA>,TV-C1,MOS,1
2,79.0,TELEVISAO OESTE BAIANO LTDA,06030116240,322647029,Barreiras,2903201,BA,-12.1013888888888333,-44.9936111111110000,A,248,<NA>,6000.0,2023-12-31,TV-C4,MOS,1
3,69.0,TELEVISAO SANTA CRUZ LTDA,06020355110,322623553,Itabuna,2914802,BA,-14.7794444444443333,-39.2622222222221666,A,248,<NA>,6000.0,2023-12-31,TV-C4,MOS,1
4,177.0,TV CABRALIA LTDA,06020354903,322623537,Itabuna,2914802,BA,-14.7833333333333333,-39.2833333333333333,B,248,<NA>,6000.0,2023-12-31,TV-C4,MOS,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30340,635.0,REDE RONDONIA DE COMUNICACAO LTDA,50444067167,<NA>,Benjamin Constant,1300607,AM,<NA>,<NA>,C,801,<NA>,5700.0,<NA>,TV-C1,MOS,1
30341,635.0,REDE RONDONIA DE COMUNICACAO LTDA,50444067248,<NA>,Codajás,1301308,AM,<NA>,<NA>,C,801,<NA>,5700.0,<NA>,TV-C1,MOS,1
30342,629.0,REDE RONDONIA DE COMUNICACAO LTDA,50444067400,<NA>,Parintins,1303403,AM,<NA>,<NA>,C,801,<NA>,5700.0,<NA>,TV-C1,MOS,1
30343,91.1,SISTEMA NOROESTE DE COMUNICACAO LTDA EPP,50407817506,1007790129,Piacatu,3537701,SP,<NA>,<NA>,C,230,<NA>,256.0,2029-01-08,FM-C4,MOS,1


In [16]:
#| export
def update_telecom(
    mongo_client: MongoClient,  # Objeto de conexão com o MongoDB
    folder: Union[str, Path],  # Pasta onde salvar os arquivos
) -> pd.DataFrame:  # DataFrame com os dados atualizados
    """Efetua a query na tabela `licenciamento` no banco mongoDB `mongo_client` e atualiza o arquivo local"""

    database = mongo_client["sms"]
    collection = database["licenciamento"]
    c = collection.find(MONGO_TELECOM, projection={k: 1.0 for k in COLS_TELECOM.keys()})
    print("[red] :warning: Executando a query na base licenciamento do Mosaico, processo demorado! :warning:")
    result = L()
    for doc in tqdm(c):
        result.append(doc)
    df = pd.DataFrame(result, dtype='string')
    df.drop("_id", axis=1, inplace=True)
    df.rename(COLS_TELECOM, axis=1, inplace=True)
    df["Designacao_Emissão"] = df.Designacao_Emissão.str.replace(",", " ")
    df["Designacao_Emissão"] = (
        df.Designacao_Emissão.str.strip().str.lstrip().str.rstrip().str.upper()
    )
    df["Designacao_Emissão"] = df.Designacao_Emissão.str.split(" ")
    df = df.explode("Designacao_Emissão")
    df.loc[df.Designacao_Emissão == "/", "Designacao_Emissão"] = ""
    df.loc[:, ["Largura_Emissão(kHz)", "Classe_Emissão"]] = df.Designacao_Emissão.apply(
        parse_bw
    ).tolist()
    df.drop("Designacao_Emissão", axis=1, inplace=True)
    _ = _save_df(df, folder, "telecom_raw")
    subset = [
        "Entidade",
        "Longitude",
        "Latitude",
        "Classe",
        "Frequência",
        "Num_Serviço",
        "Largura_Emissão(kHz)",
        "Classe_Emissão",
    ]
    df.dropna(subset=subset, axis=0, inplace=True)
    df_sub = (
        df[~df.duplicated(subset=subset, keep="first")].reset_index(drop=True).copy()
    )
    df_sub["Multiplicidade"] = (
        df.groupby(subset, sort=False).count()["Número_Estação"]
    ).tolist()
    df_sub["Status"] = "L"
    df_sub["Fonte"] = "MOS"
    del df
    gc.collect()
    df_sub = df_sub.reset_index()
    df_sub = df_sub.loc[:, COLUNAS]
    return _save_df(df_sub, folder, "telecom")

In [17]:
#| eval: false
telecom = update_telecom(mongo_client, folder)
telecom

 ⚠ Executando a query na base licenciamento do Mosaico, processo demorado! ⚠

0it [00:00, ?it/s]

,Frequência,Entidade,Fistel,Número_Estação,Município,Código_Município,UF,Latitude,Longitude,Classe,Num_Serviço,Classe_Emissão,Largura_Emissão(kHz),Validade_RF,Status,Fonte,Multiplicidade
0,173.075,TRACKER DO BRASIL LTDA,50004466071,505805553,Mairiporã,3528502,SP,-23.4030555555555,-46.63583333333333,FB,011,-1,-1,2034-12-23,L,MOS,1
1,173.075,TRACKER DO BRASIL LTDA,50004466071,684490021,Belo Horizonte,3106200,MG,-19.793333333333333,-43.968888888888834,FB,011,-1,-1,2034-12-23,L,MOS,1
2,173.075,TRACKER DO BRASIL LTDA,50004466071,684587017,Tubarão,4218707,SC,-28.520833333333332,-48.98527777777767,FB,011,-1,-1,2034-12-23,L,MOS,1
3,173.075,TRACKER DO BRASIL LTDA,50004466071,684610094,Florianópolis,4205407,SC,-27.588611111111,-48.533611111111,FB,011,-1,-1,2034-12-23,L,MOS,1
4,173.075,TRACKER DO BRASIL LTDA,50004466071,684649900,João Monlevade,3136207,MG,-19.868888888888833,-43.193333333333335,FB,011,-1,-1,2034-12-23,L,MOS,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
728019,2072.5,CENTRO GESTOR E OPERACIONAL DO SISTEMA DE PROT...,50438537947,1014591861,Manaus,1302603,AM,-2.8912583333333,-59.969280555556,TC,183,XXX,100000.0,2041-06-01,L,MOS,1
728020,8150.0,CENTRO GESTOR E OPERACIONAL DO SISTEMA DE PROT...,50438537947,1014591861,Manaus,1302603,AM,-2.8912583333333,-59.969280555556,TC,183,XXX,700000.0,2041-06-01,L,MOS,1
728021,3330.0,VIDEOCOM BRASIL LTDA,50013743597,683898060,,3304557,RJ,-22.91,-43.174167,TX,820,F9F,18000.0,2022-12-18,L,MOS,1
728022,3370.0,VIDEOCOM BRASIL LTDA,50013743597,683964275,,3304557,RJ,-22.950556,-43.21,FX,820,F9F,18000.0,2022-12-18,L,MOS,1


In [18]:
#| export
def valida_coords(
    conn: pyodbc.Connection,  # Objeto de conexão de banco
    row: pd.Series,  # Linha de um DataFrame
) -> tuple:  # DataFrame com dados do município
    """Valida os dados de coordenadas e município em `row` no polígono dos municípios em banco corporativo"""

    sql = SQL_VALIDA_COORD.format(
        row["Longitude"], row["Latitude"], row["Código_Município"]
    )
    crsr = conn.cursor()
    crsr.execute(sql)
    result = crsr.fetchone()
    if result is None:
        return (row["Município"], row["Longitude"], row["Latitude"], -1)
    elif result.COORD_VALIDA == 1:
        return result
    else:
        return (
            result.NO_MUNICIPIO,
            result.NU_LONGITUDE,
            result.NU_LATITUDE,
            result.COORD_VALIDA,
        )

In [19]:
# | export
def update_base(
    conn: pyodbc.Connection,  # Objeto de conexão de banco
    clientMongoDB: MongoClient,  # Ojeto de conexão com o MongoDB
    folder: Union[str, Path],  # Pasta onde salvar os arquivos
) -> pd.DataFrame:  # DataFrame com os dados atualizados
    # sourcery skip: use-fstring-for-concatenation
    """Wrapper que atualiza opcionalmente lê e atualiza as 4 bases indicadas anteriormente, as combina e salva o arquivo consolidado na folder `folder`"""
    stel = update_stel(
        conn,
        folder,
    )
    radcom = update_radcom(conn, folder)
    mosaico = update_mosaico(clientMongoDB, folder)
    telecom = update_telecom(clientMongoDB, folder)

    rd = (
        pd.concat([mosaico, radcom, stel, telecom])
        .sort_values(["Frequência", "Latitude", "Longitude"])
        .reset_index(drop=True)
    )
    rd.loc[:, ["Latitude", "Longitude"]] = rd.loc[:, ["Latitude", "Longitude"]].fillna(
        "-1"
    )  # Validando Coordenadas
    rd["Coords_Valida"] = pd.NA
    rd[["Município", "Longitude", "Latitude", "Coords_Valida"]] = rd.apply(
        lambda row: pd.Series(list(valida_coords(conn, row))), axis=1
    )
    rd = rd.drop(rd[rd.Coords_Valida == -1].index)
    return _save_df(rd, folder, "base")

In [20]:
from extracao.reading import *

#| eval: false
base = update_base(conn, mongo_client, folder)
base

,Frequência,Entidade,Fistel,Número_Estação,Município,Código_Município,UF,Latitude,Longitude,Classe,Num_Serviço,Classe_Emissão,Largura_Emissão(kHz),Validade_RF,Status,Fonte,Multiplicidade
0,0.028,FURNAS CENTRAIS ELETRICAS S A,01030052263,1557670,,3303500,RJ,-22.662778,-43.476389,OP,019,J9E,8.0,2033-08-17,L,MOS,1
1,0.03,FURNAS CENTRAIS ELETRICAS S A,01030052263,859966,,3103751,MG,-18.41,-49.1,OP,019,J3E,1.0,2033-08-17,L,MOS,1
2,0.03,FURNAS CENTRAIS ELETRICAS S A,01030052263,859753,,3509502,SP,-22.774167,-47.004444,OP,019,J3E,1.0,2033-08-17,L,MOS,1
3,0.03,FURNAS CENTRAIS ELETRICAS S A,01030052263,859761,,3304557,RJ,-22.926667,-43.265,OP,019,J3E,0.5,2033-08-17,L,MOS,1
4,0.03,FURNAS CENTRAIS ELETRICAS S A,01030052263,1557823,,3550308,SP,-23.441667,-46.590833,OP,019,J3E,1.0,2033-08-17,L,MOS,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
834490,991.0,EMPRESA BRASILEIRA DE INFRA-ESTRUTURA AEROPORT...,50411187015,1000823749,Uberlândia,3170206,MG,-18.872222222222167,-48.221111111111,RC,108,-1,-1.0,2034-09-17,L,MOS,1
834491,991.0,EMPRESA BRASILEIRA DE INFRA-ESTRUTURA AEROPORT...,50411187015,1000822734,Belo Horizonte,3106200,MG,-19.855277777777665,-43.93722222222217,RC,108,-1,-1.0,2034-09-17,L,MOS,1
834492,991.0,EMPRESA BRASILEIRA DE INFRA-ESTRUTURA AEROPORT...,50411187015,1000823218,Santarém,1506807,PA,-2.4227777777776667,-54.78916666666667,RC,108,-1,-1.0,2034-09-17,L,MOS,1
834493,991.0,EMPRESA BRASILEIRA DE INFRA-ESTRUTURA AEROPORT...,50411187015,1000823846,Vitória,3205309,ES,-20.266944444444334,-40.292777777777665,RC,108,-1,-1.0,2034-09-17,L,MOS,1
